In [ ]:
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import numpy as np
import cv2
from keras.models import load_model
import tensorflow as tf
import tensorflow.keras as k
from keras.layers import Input
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy

In [ ]:
data_dir = 'data'

# Dense Net

In [ ]:
#Loading the data
data= tf.keras.utils.image_dataset_from_directory(data_dir, seed=42, image_size=(224, 224), label_mode='binary')

#scaling the data
scaled_data = data.map(lambda x, y:(x/255, y))

#Defining split sizes and splitting the data
train_size = int(len(data)*.7)
test_size = int(len(data)*.1)+2
val_size = int(len(data)*.2)
train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size+val_size).take(test_size)
print(train_size+ test_size+ val_size)

In [ ]:
#Initializing the densenet model
img_inputs = Input(shape=(224, 224, 3), name='data')
base_model = k.applications.DenseNet121(include_top=False, input_tensor=img_inputs, input_shape=(224,224,3), pooling='max')

#Making the base model layers Non-Trainable
base_model.trainable = False

In [ ]:
model = k.Sequential()
model.add(base_model)

model.add(k.layers.Flatten())
model.add(k.layers.BatchNormalization())
model.add(k.layers.Dense(256, activation='relu'))
model.add(k.layers.Dropout(0.6))
model.add(k.layers.BatchNormalization())
model.add(k.layers.Dense(128, activation='relu'))
model.add(k.layers.Dropout(0.4))
model.add(k.layers.BatchNormalization())
model.add(k.layers.Dense(64, activation='relu'))
model.add(k.layers.Dense(2, activation='softmax'))

model.compile('adam', loss = 'categorical_crossentropy', metrics=['accuracy'])

epochs = 10

In [ ]:
#Training
history = model.fit(train, epochs=epochs, validation_data=val, verbose=1)

In [ ]:
# Saving the model
model.save('models/densenet.h5', save_format='h5')

In [ ]:
pre = Precision()
recall = Recall()
Acc = BinaryAccuracy()

loaded_densenet_model = load_model('models/densenet.h5')

for Batch in test.as_numpy_iterator():
    x, y = Batch
    y_pred = loaded_densenet_model.predict(x, verbose=0, use_multiprocessing=True)
    pre.update_state(y, y_pred)
    recall.update_state(y, y_pred)
    Acc.update_state(y, y_pred)

print(f"precision:{(pre.result()*100):.2f}, recall:{(recall.result()*100):.2f}, accuracy:{(Acc.result()*100):.2f}")

# **InceptionV3**

In [ ]:
#Loading and Scaling the data
data= tf.keras.utils.image_dataset_from_directory(data_dir, seed=42, image_size=(299, 299), label_mode='binary')
scaled_data = data.map(lambda x, y:(x/255, y))

In [ ]:
train_size = int(len(data)*.7)
test_size = int(len(data)*.1)+2
val_size = int(len(data)*.2)

train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size+val_size).take(test_size)

print(train_size+ test_size+ val_size)

In [ ]:
img_inputs = Input(shape=(299, 299, 3), name='data')
inc_base_model = k.applications.InceptionV3(include_top=False, input_tensor=img_inputs, input_shape=(299,299,3))
inc_base_model.trainable=False

In [ ]:
inc_model = k.Sequential()
inc_model.add(inc_base_model)

inc_model.add(k.layers.Flatten())
inc_model.add(k.layers.BatchNormalization())

inc_model.add(k.layers.Dense(512, activation='relu'))
inc_model.add(k.layers.Dropout(0.3))
inc_model.add(k.layers.BatchNormalization())

inc_model.add(k.layers.Dense(1, activation='sigmoid'))
inc_model.compile('adam', loss = tf.losses.BinaryCrossentropy(), metrics=['accuracy'])

epoch=10

In [ ]:
inc_history = inc_model.fit(train, epochs=epochs, validation_data=val, verbose = 1)

In [ ]:
inc_model.save('models/ncV3.h5', save_format='h5')

In [ ]:
pre = Precision()
recall = Recall()
Acc = BinaryAccuracy()

loaded_inc_model = load_model('models/incv3')

for Batch in test.as_numpy_iterator():
    x, y = Batch
    y_pred = loaded_inc_model.predict(x, verbose=0, use_multiprocessing=True)
    pre.update_state(y, y_pred)
    recall.update_state(y, y_pred)
    Acc.update_state(y, y_pred)

print(f"precision:{(pre.result()*100):.2f}, recall:{(recall.result()*100):.2f}, accuracy:{(Acc.result()*100):.2f}")

# **VGG16**

In [ ]:
# !pip install -q tensorflow-io

In [ ]:
data= tf.keras.utils.image_dataset_from_directory(data_dir, seed=42, image_size=(224, 224), label_mode='binary')

In [ ]:
import tensorflow_io as tfio
scaled_data = data.map(lambda x, y:(tfio.experimental.color.rgb_to_bgr(x), y))
scaled_data = scaled_data.map(lambda x, y:(x/255, y))

In [ ]:
train_size = int(len(data)*.7)
test_size = int(len(data)*.1)+2
val_size = int(len(data)*.2)
print(train_size+ test_size+ val_size)
train = scaled_data.take(train_size)
val = scaled_data.skip(train_size).take(val_size)
test = scaled_data.skip(train_size+val_size).take(test_size)

In [ ]:
img_inputs = Input(shape=(224, 224, 3), name='data')
vgg_base_model = k.applications.vgg16.VGG16(include_top=False, input_tensor=img_inputs, input_shape=(224,224,3))
vgg_base_model.trainable = False

In [ ]:
vgg_model = k.Sequential()
vgg_model.add(vgg_base_model)
vgg_model.add(k.layers.Flatten())
vgg_model.add(k.layers.BatchNormalization())
vgg_model.add(k.layers.Dense(256, activation='relu'))
vgg_model.add(k.layers.Dropout(0.4))
vgg_model.add(k.layers.BatchNormalization())
vgg_model.add(k.layers.Dense(1, activation='sigmoid'))

In [ ]:
epochs=10

In [ ]:
vgg_history = vgg_model.fit(train, epochs=epochs, validation_data=val, verbose = 1)

In [ ]:
vgg_model.save('vgg16.h5', save_format='h5', overwrite=True)

In [ ]:

pre = Precision()
recall = Recall()
Acc = BinaryAccuracy()

vgg_loaded_model = load_model('models/vgg16_v2.h5')

for Batch in test.as_numpy_iterator():
    x, y = Batch
    y_pred = vgg_loaded_model.predict(x, verbose=0, use_multiprocessing=True)
    pre.update_state(y, y_pred)
    recall.update_state(y, y_pred)
    Acc.update_state(y, y_pred)

print(f"precision:{(pre.result()*100):.2f}, recall:{(recall.result()*100):.2f}, accuracy:{(Acc.result()*100):.2f}")